<a href="https://colab.research.google.com/github/Ahangren/bert_project02/blob/branchname1/zhangren.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer,BertConfig
from transformers import BertForSequenceClassification,get_linear_schedule_with_warmup
# from transformers import AdamW
from torch.optim import AdamW
from tqdm import tqdm,trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt



device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')



df=pd.read_csv("/content/drive/MyDrive/transformersshijian/betr实践/Transformers-for-NLP-2nd-Edition/Chapter03/in_domain_train.tsv",
               delimiter='\t',header=None,names=['sentence_source','label','label_notes','sentence'])

# /content/gdrive/MyDrive/transformersshijian/betr实践/Transformers-for-NLP-2nd-Edition/Chapter03/out_of_domain_dev.tsv

sentences=df.sentence.values
print(sentences)


["our friends wo n't buy this analysis , let alone the next one we propose ."
 "one more pseudo generalization and i 'm giving up ."
 "one more pseudo generalization or i 'm giving up ." ...
 'it is easy to slay the gorgon .'
 'i had the strangest feeling that i knew you .'
 'what all did you get for christmas ?']


In [7]:
sentences=["[CLS]"+ sentence+"[SEP]" for sentence in sentences]
labels=df.label.values
# print(sentences)

In [8]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_texts=[tokenizer.tokenize(str(sent)) for sent in sentences]
print("Tokenize the first sentence: ")
print(tokenizer_texts[0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Tokenize the first sentence: 
['[CLS]', 'our', 'friends', 'wo', 'n', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.', '[SEP]']


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN=128
input_ids=[tokenizer.convert_tokens_to_ids(x) for x in tokenizer_texts]
print(input_ids[0])
input_ids=pad_sequences(input_ids,maxlen=MAX_LEN,dtype='long',truncating='post',padding='post')
print(input_ids[0])


[101, 2256, 2814, 24185, 1050, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]
[  101  2256  2814 24185  1050  1005  1056  4965  2023  4106  1010  2292
  2894  1996  2279  2028  2057 16599  1012   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [10]:
attention_masks=[]
for seq in input_ids:
  seq_mask=[float(i>0) for i in seq]
  attention_masks.append(seq_mask)
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [11]:
# 将数据拆分成训练集和测试集
train_inputs,validation_inputs,train_labels,validation_label=train_test_split(input_ids,labels,random_state=2025,test_size=0.1)
train_masks,validation_masks,train_labels_masks,validation_label_masks=train_test_split(attention_masks,input_ids,random_state=2025,test_size=0.1)

# 将所有的数据拆分成torch张量
train_inputs=torch.tensor(train_inputs)
validation_inputs=torch.tensor(validation_inputs)
train_labels=torch.tensor(train_labels)
validation_label=torch.tensor(validation_label)
train_masks=torch.tensor(train_masks)
validation_masks=torch.tensor(validation_masks)
train_labels_masks=torch.tensor(train_labels_masks)
validation_label_masks=torch.tensor(validation_label_masks)
print(train_masks)


tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.]])


In [12]:
"""
选择批量大小并创建迭代器
如果一股脑地将所有数据都喂进机器，会导致机器因为内存不足而崩溃。所以需
要将数据一批一批地喂给机器。这里将把批量大小(batch size)设置为 32 并创建迭代
器。然后将迭代器与 torch的 DataLoader 相结合，以批量训练大量数据集，以免导致
机器因为内存不足而崩溃：
"""

batch_size=32
# 训练数据迭代器
train_data=TensorDataset(train_inputs,train_masks,train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)
# 测试数据迭代器

validation_data=TensorDataset(validation_inputs,validation_masks,validation_label)
validation_sampler=RandomSampler(train_data)
validation_dataloader=DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)



In [13]:
#BERT模型配置：这里我们初始化一个不区分大小写的BERT配置
try:
  import transformers
except:
  print("installing transformers")
from transformers import BertModel,BertConfig
configuration=BertConfig()

model=BertModel(configuration)
configuration=model.config
print(configuration)

BertConfig {
  "_attn_implementation_autoset": true,
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.50.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [14]:
"""
attention probs_dropout_prob:对注意力概率应用的 dropout 率，这里设置为
0.1。
● hidden_act;编码器中的非线性激活函数，这里使用 gelu。gelu 是高斯误差线
性单位(Gaussian Eror Linear Units)激活函数的简称，它对输入按幅度加权，
使其成为非线性。
● hidden_dropout_prob:应用于全连接层的 dropout 概率。嵌入、编码器和汇聚
器层中都有全连接。输出不总是对序列内容的良好反映。汇聚隐藏状态的序
第3章 微调BERT 模型
列可改善输出序列。这里设置为0.1。
● hidden_size:编码器层的维度，也是汇聚层的维度，这里设置为768。
● initializer_range:初始化权重矩阵时的标准偏差值，这里设置为0.02。
· intermediate_size:编码器前馈层的维度，这里设置为3072。
● layer_norm_eps:是层规范化层的 epsilon 值，这里设置为le-12。
● max_position_embeddings:模型使用的最大长度，这里设置为512。
● model_type:模型的名称，这里设置为 bert。
● numattention_heads:注意力头数，这里设置为12。
· num_hidden_layers:层数，这里设置为12。
● pad_tokenid:使用0作为填充词元的HD,以避免对填充词元进行训练。
57
 · type_vocab_size:token_type_ids的大小用于标识序列。例如，“the dog[SEP]
 The cat.[SEP]”可用词元 ID [0,0,0,1,1,1]表示。
· vocab_size:模型用于表示 input_ids 的不同词元数量。换句话说，这是模型
可以识别和处理的不同词元或单词的总数。在训练过程中，模型会根据给定
的词表将文本输入转换为对应的词元序列，其中包含的词元数量是
vocab_size。通过使用这个词表，模型能够理解和表示更广泛的语言特征。这
里设置为 30522。
讲解完这些参数后，接下来将加载预训练模型。
"""

'\nattention probs_dropout_prob:对注意力概率应用的 dropout 率，这里设置为\n0.1。\n● hidden_act;编码器中的非线性激活函数，这里使用 gelu。gelu 是高斯误差线\n性单位(Gaussian Eror Linear Units)激活函数的简称，它对输入按幅度加权，\n使其成为非线性。\n● hidden_dropout_prob:应用于全连接层的 dropout 概率。嵌入、编码器和汇聚\n器层中都有全连接。输出不总是对序列内容的良好反映。汇聚隐藏状态的序\n第3章 微调BERT 模型\n列可改善输出序列。这里设置为0.1。\n● hidden_size:编码器层的维度，也是汇聚层的维度，这里设置为768。\n● initializer_range:初始化权重矩阵时的标准偏差值，这里设置为0.02。\n· intermediate_size:编码器前馈层的维度，这里设置为3072。\n● layer_norm_eps:是层规范化层的 epsilon 值，这里设置为le-12。\n● max_position_embeddings:模型使用的最大长度，这里设置为512。\n● model_type:模型的名称，这里设置为 bert。\n● numattention_heads:注意力头数，这里设置为12。\n· num_hidden_layers:层数，这里设置为12。\n● pad_tokenid:使用0作为填充词元的HD,以避免对填充词元进行训练。\n57\n · type_vocab_size:token_type_ids的大小用于标识序列。例如，“the dog[SEP]\n The cat.[SEP]”可用词元 ID [0,0,0,1,1,1]表示。\n· vocab_size:模型用于表示 input_ids 的不同词元数量。换句话说，这是模型\n可以识别和处理的不同词元或单词的总数。在训练过程中，模型会根据给定\n的词表将文本输入转换为对应的词元序列，其中包含的词元数量是\nvocab_size。通过使用这个词表，模型能够理解和表示更广泛的语言特征。这\n里设置为 30522。\n讲解完这些参数后，接下来将加载预训练模型。\n'

In [15]:
model=BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
model=nn.DataParallel(model)
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=7

In [28]:
# BertForSequenceClassification(
#  (bert):BertMòdel(
#  (enbeddings): BertBmbeddingat
#  (word_embeddings):.Embedding(30522;76
#  padding
#  (position_embeddings):Embedding(512,768)
#  (token_typeembeddinge):Embedding(2;7681
#  (LayerNorm):BerttayerNorm()
#  (dropout):Dropoutip-0.1,inpiacesFalse
#  )

#   (encoder):BertEncoder(                             (encoder):BertEncoder(
#  (layer):ModuieList(
#  (0):BertLayer(,
#  (attention):Bertattentlon(
#  (self): BertSelfAttention(
#  bias=True
#  bias-True)
#  613
#  (query):linearsin_featires-768, out_features=768,
#  (key):Linear(in_features=768, out_features-768,biaseTrue)
#  (value):Linear(tin_features=768,out_features=768,bias=True)
#  (dropout):Dropout(p=0.1, inplace=False)
#  toutput); BertselfOutput (
#  :(dense):Linear(in_teatures=768, out_featuregw76B,
#  (LayerNorm):BertlayerNdm()
#  (drapout);.Dropout(p-0.1, inplace=False)
#  iintermediatel:BertIntermadiate(
#  (denee): Uinear(in_features=768; out_features-3072,bias=True)
#  (output);Bertoutput (
#  .(dense):Linear(in_teatured=3072,out_features=768,biag=True)
#  (tayerNorm); BertLayerNorm()
#  tdropout):Dropout(p=0.1,inplace-False)
#  (1):BertLayer(
#  tattention):Bertattentiont
#  (aelf):BertSelfAttention(
#  {query}:Linearfin_features-768,out_features=768,
#  bias=True)
#  (key):Linear(in_features=768,out_features=768,bias=True).
#  (value):Linear(in_features=768, out Features=768,
#  bias=Trua).
#  (dropout):Dropout(p=0.1, inplace-False)
#  (output): BertseIfoutput i
#  (dense):Lineartin_features-768,outfeatureg=768,
#  bias=True)
#  (LayerNorm): BertLayervorm()
#  (dropout):Dropout(p=0.1,inplace=False)))
# (intermediate): BertIatermediate(
#  (dense):Linear(in_features=768, out_features=3072,bias=True)
#  (output):BertOutput (
#  .(dense):Linear(in_featureg=3072,out_features=768, bias=True)
#  .(LayerNorm): BertLayerNorm()
#  (dropout):Dropout (p=0,1,inplace=False))

SyntaxError: invalid syntax (<ipython-input-28-b2d79cb2ecd1>, line 2)

In [36]:
# 优化器分组参数
"""
现在将为模型的参数初始化优化器。在进行模型微调的过程中，首先需要初始化
预训练模型已学到的参数值。
微调一个预训练模型时，通常会使用之前在大规模数据上训练好的模型作为初始
模型。这些预训练模型已通过大量数据和计算资源进行了训练，学到了很多有用的特
征表示和参数权重。因此，我们希望在微调过程中保留这些已经学到的参数值，而不
是重新随机初始化它们。
所以，程序会使用预训练模型的参数值来初始化优化器，以便在微调过程中更好
地利用这些已经学到的参数。这样可以加快模型收敛速度并提高微调效果；

"""
param_optimizer=list(model.named_parameters())

no_decay=['bias','LayerNorm.weight']
optimizer_grouped_parametes=[
    {'params':[p for n,p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate':0.1},
    {"params":[p for n,p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate':0.0}
]

"""
在深度学习中，过滤掉不包含 'bias'、'gamma'、'beta' 的参数，通常是为了 ​对不同类型的参数应用不同的优化策略。例如，避免对这些参数应用权重衰减（weight decay），以提高模型训练的稳定性和效果。
归一化层的 gamma/beta：用于缩放和偏移归一化后的值，正则化会干扰其学习过程。
"""


In [46]:
optimizer=AdamW(optimizer_grouped_parametes,lr=2e-5)
def float_accuracy(preds,labels):
  pred_flat=np.argmax(preds,axis=1).floatten()
  labels_flat=labels.floatten()
  return np.sum(pred_flat==labels_flat)/len(labels_flat)


In [49]:
"""
3.2.17 训练循环
我们的训练循环将遵循标准的学习过程。轮数(epochs)设置为 4,并将绘制损失和
准确率的度量值。训练循环使用 dataloader 来加载和训练批量。我们将对训练过程进
行度量和评估。
首先初始化 train_loss_set(用于存储损失和准确率的数值，以便后续绘图)。然后
开始训练每一轮，并运行标准的训练循环，如下所示：
"""
# t=[]
# train_loss_set=[]
# epochs=4

# for _ in trange(epochs,desc="Epoch"):
#   logits=model.train()
#   tmp_eval_accuracy = float_accuracy(logits,label_ids)
#   eval_accuracy += tmp_eval_accuracy
#   nb_eval_steps += 1
#   print("valIdation Accuracy:()".format (teval_accuracy/nb_eval_steps))



'\n3.2.17 训练循环\n我们的训练循环将遵循标准的学习过程。轮数(epochs)设置为 4,并将绘制损失和\n准确率的度量值。训练循环使用 dataloader 来加载和训练批量。我们将对训练过程进\n行度量和评估。\n首先初始化 train_loss_set(用于存储损失和准确率的数值，以便后续绘图)。然后\n开始训练每一轮，并运行标准的训练循环，如下所示：\n'